In [25]:
import psycopg2
import psycopg2.extras
import csv
import os
import pandas as pd
from config2 import params2


# SPRINT 6
conn = psycopg2.connect(**params2)


In [26]:
# opening the csv containing bootcamps data
# to insert the data into tables in the database

df = pd.read_csv("./../data/combined_skills.csv")

In [27]:
# from the dataset, obtain the different school_names
school_set = set(df["school_name"])

# connect to the database
conn = psycopg2.connect(**params2)

# insert into bootcamps_edu.School table
cursor = conn.cursor()
for sc in school_set:
    cmd = f"INSERT INTO bootcamps_edu.School (school_name) VALUES ('{sc}')"
    cursor.execute(cmd)

conn.commit()
cursor.execute("select * from bootcamps_edu.School")
school_lst = cursor.fetchall()
conn.commit()
conn.close()

print(school_lst)


[(1, 'Code Chrysalis'), (2, 'Turing School of Software and Design'), (3, 'Tech Elevator'), (4, 'Code Platoon'), (5, 'The Tech Academy'), (6, 'Codesmith'), (7, 'Juno College of Technology'), (8, 'Launch Academy'), (9, 'Codeup')]


In [28]:
# insert into bootcamps_edu.Program table
# from the dataset, obtain the different programs
program_name_set = set(df["program_name"])

# connect to the database
conn = psycopg2.connect(**params2)

# insert into bootcamps_edu.School table
cursor = conn.cursor()
for pr in program_name_set:
    cmd = f"INSERT INTO bootcamps_edu.Program (program_name) VALUES ('{pr}')"
    cursor.execute(cmd)
    #print(cmd)

conn.commit()
cursor.execute("select * from bootcamps_edu.Program")
program_lst = cursor.fetchall()
conn.commit()
conn.close()

print(program_lst)



[(1, 'Front-End Web Developer BootCamp'), (2, 'Back End Engineering'), (3, 'Data Analytics Bootcamp'), (4, 'Software Developer BootCamp'), (5, 'C# and .NET Boot Camp'), (6, 'Python Developer BootCamp'), (7, 'Full-Stack Web Development'), (8, 'Data Science BootCamp'), (9, 'Front End Engineering'), (10, 'Full-Stack Software Development'), (11, 'Full-Stack Web Development-Java'), (12, 'Web Development Immersive'), (13, 'Fundamentals of Web Development'), (14, 'Part-Time Full-Stack Web Development'), (15, 'The Immersive Bootcamp')]


In [29]:
# insert states
# from the dataset, obtain the different states
state_set = set(df["state"])

# connect to the database
conn = psycopg2.connect(**params2)

# insert into bootcamps_edu.School table
cursor = conn.cursor()
for st in state_set:
    cmd = f"INSERT INTO bootcamps_edu.State (name) VALUES ('{st}')"
    cursor.execute(cmd)
    #print(cmd)

conn.commit()
cursor.execute("select * from bootcamps_edu.State")
state_lst = cursor.fetchall()
conn.commit()
conn.close()

print(state_lst)



[(1, 'Illinois'), (2, 'Ohio'), (3, 'Texas'), (4, 'New York'), (5, 'Massachusetts'), (6, 'Online'), (7, 'Pennsylvania'), (8, 'Japan'), (9, 'Delaware'), (10, 'Canada'), (11, 'Utah'), (12, 'California')]


In [30]:
# table bootcamps_edu.Location
#	id int PRIMARY KEY GENERATED BY DEFAULT AS IDENTITY,
#	name varchar(50) not null,
#	state_id int not null REFERENCES bootcamps_edu.State (id)

# in the csv we have location (city), and state name
# we need to relate location (city) from csv, and state name from csv
# then get state id from table, using the state name
# then insert into table Location: location name and state_id

# get campus_location and state from csv (df)
locations_df = df[['campus_location', 'state']]

locations_lst = locations_df.values.tolist()

# connect to the database
conn = psycopg2.connect(**params2)

# get the states
cursor = conn.cursor()
cursor.execute("select * from bootcamps_edu.State")
state_lst = cursor.fetchall()
conn.commit()

states_df = pd.DataFrame(data=state_lst,columns=["id", "state"])

#states_df

location_list = list()

for l in locations_lst:
    id = states_df[states_df["state"] == l[1]]["id"]
    location_list.append([l[0], int(id.values)])

unique_location_list = []
[unique_location_list.append(x) for x in location_list if x not in unique_location_list]


for l in unique_location_list:
    cmd = f"INSERT INTO bootcamps_edu.Location (name, state_id) VALUES ('{l[0]}', {l[1]})"
    cursor.execute(cmd)

conn.commit()
cursor.execute("select * from bootcamps_edu.Location")
location_lst = cursor.fetchall()
conn.commit()
conn.close()

location_lst



[(1, 'Online', 6),
 (2, 'Dallas', 3),
 (3, 'Chicago', 1),
 (4, 'Pittsburgh', 7),
 (5, 'San Antonio', 3),
 (6, 'Los Angeles', 12),
 (7, 'Remote', 6),
 (8, 'Cleveland', 2),
 (9, 'Columbus', 2),
 (10, 'Cincinnati', 2),
 (11, 'Philadelphia', 7),
 (12, 'Toronto', 10),
 (13, 'New York', 4),
 (14, 'Salt Lake City', 11),
 (15, 'Tokyo', 8),
 (16, 'San Antonio - Vogue', 3),
 (17, 'Boston', 5),
 (18, 'Wilmington', 9)]

In [31]:
# insert school_program_location
#   TABLE DEFINITION:
#	id int PRIMARY KEY GENERATED BY DEFAULT AS IDENTITY,
#	school_id int not null REFERENCES bootcamps_edu.School (id),
#	program_id int not null REFERENCES bootcamps_edu.Program (id),
#	location_id int not null REFERENCES bootcamps_edu.Location (id)

# get all schools, locations, and programs from original df
school_location_program_df = df[['school_name', 'campus_location', 'program_name']]

school_location_program_lst = school_location_program_df.values.tolist()

# connect to the database to get programs and locations
conn = psycopg2.connect(**params2)
cursor = conn.cursor()

# get schools ids and names from the school table in the db
cursor.execute("select * from bootcamps_edu.School")
school_lst = cursor.fetchall()
conn.commit()

schools_df = pd.DataFrame(data=school_lst,columns=["id", "school_name"])

# get programs ids and names from the program table in the db
cursor.execute("select * from bootcamps_edu.Program")
program_lst = cursor.fetchall()
conn.commit()

programs_df = pd.DataFrame(data=program_lst,columns=["id", "program_name"])

# get locations ids and names from the location table in the db
cursor.execute("select * from bootcamps_edu.Location")
location_lst = cursor.fetchall()
conn.commit()

locations_df = pd.DataFrame(data=location_lst,columns=["id", "name", "state_id"])

# begin unique values
school_program_location_lst = list()

for lp in school_location_program_lst:
    school_id = schools_df[schools_df["school_name"] == lp[0]]["id"]
    school_id_int = int(school_id.values)
    location_id = locations_df[locations_df["name"] == lp[1]]["id"]
    location_id_int = int(location_id.values)
    program_id = programs_df[programs_df["program_name"] == lp[2]]["id"]
    program_id_int = int(program_id.values)
    school_program_location_lst.append([school_id_int, program_id_int, location_id_int])

unique_school_program_location_lst = []
[unique_school_program_location_lst.append(x) for x in school_program_location_lst if x not in unique_school_program_location_lst]
# end unique values

# insert program_id and location_id into the program_location table
for lp in unique_school_program_location_lst:
    cmd = f"INSERT INTO bootcamps_edu.School_Program_Location (school_id, program_id, location_id) VALUES ({lp[0]}, {lp[1]}, {lp[2]})"
    cursor.execute(cmd)

conn.commit()
cursor.execute("select * from bootcamps_edu.School_Program_Location")
db_school_program_location_lst = cursor.fetchall()
conn.commit()
conn.close()

db_school_program_location_lst




[(1, 5, 6, 1),
 (2, 5, 8, 1),
 (3, 2, 2, 1),
 (4, 9, 7, 2),
 (5, 4, 7, 3),
 (6, 3, 10, 4),
 (7, 9, 8, 5),
 (8, 6, 7, 6),
 (9, 3, 10, 7),
 (10, 3, 10, 8),
 (11, 3, 10, 9),
 (12, 9, 7, 5),
 (13, 9, 11, 5),
 (14, 3, 10, 10),
 (15, 6, 7, 7),
 (16, 3, 10, 11),
 (17, 2, 9, 1),
 (18, 7, 12, 12),
 (19, 5, 4, 1),
 (20, 5, 1, 1),
 (21, 6, 7, 13),
 (22, 5, 4, 14),
 (23, 1, 15, 15),
 (24, 9, 7, 16),
 (25, 9, 11, 2),
 (26, 8, 13, 17),
 (27, 9, 8, 16),
 (28, 5, 5, 1),
 (29, 6, 14, 7),
 (30, 7, 3, 12),
 (31, 3, 10, 18)]

In [33]:
# insert program_date
#table bootcamps_edu.Program_Date
#	id int PRIMARY KEY GENERATED BY DEFAULT AS IDENTITY,
#	school_program_location_id int not null REFERENCES bootcamps_edu.Program_Location (id),
#	start_date date not null,
#	end_date date not null,
#	graduation_rate decimal not null,
#	median_salary_180days decimal not null

# go through the original csv 
# for each row in the csv, get the corresponding id from the school_program_location table
#    insert the id and combine with the detailed data (start_date, end_date, grad_rate, median_salary)

# connect to the database
conn = psycopg2.connect(**params2)
cursor = conn.cursor()

df_to_list = df.values.tolist()

for r in df_to_list:
    start_date = r[36]
    end_date = r[37]
    graduation_rate = r[40]
    median_salary_180days = str(r[49]).replace(",", "")

    school = r[0]
    cmd = f"select * from bootcamps_edu.School where school_name = '{r[0]}'"
    cursor.execute(cmd)
    school_lst = cursor.fetchall()
    school_id = school_lst[0][0]

    location = r[1]
    cmd = f"select * from bootcamps_edu.Location where name = '{r[1]}'"
    cursor.execute(cmd)
    location_lst = cursor.fetchall()
    location_id = location_lst[0][0]

    program = r[2]
    cmd = f"select * from bootcamps_edu.Program where program_name = '{r[2]}'"
    cursor.execute(cmd)
    program_lst = cursor.fetchall()
    program_id = program_lst[0][0]

    program_cost = str(r[3]).replace(",", "")
    course_length = r[38]

    cmd = f"""select * 
    from bootcamps_edu.School_Program_Location 
    where school_id = {school_id} and program_id = {program_id} and location_id = {location_id}"""
    cursor.execute(cmd)
    spl_lst = cursor.fetchall()
    spl_id = spl_lst[0][0]


    cmd = f"""INSERT INTO bootcamps_edu.Program_Date 
    (school_program_location_id, start_date, end_date, course_length, 
    program_cost, graduation_rate, median_salary_180days) 
    VALUES ({spl_id}, '{start_date}', '{end_date}', '{course_length}',
    {program_cost}, {graduation_rate}, {median_salary_180days})
    """
    print(cmd)
    cursor.execute(cmd)




conn.commit()
conn.close()

#    cmd = f"select * from bootcamps.School_Program_Location where school_name = and campus_location = pro  (school_name) VALUES ('{sc}')"
#    cursor.execute("select * from bootcamps_edu.School")
#    school_lst = cursor.fetchall()

#conn.commit()
#conn.close()


INSERT INTO bootcamps_edu.Program_Date 
    (school_program_location_id, start_date, end_date, course_length, 
    program_cost, graduation_rate, median_salary_180days) 
    VALUES (1, '7/1/2021', '12/31/2021', 'Self-paced',
    9540.00, 82.4, 65000)
    
INSERT INTO bootcamps_edu.Program_Date 
    (school_program_location_id, start_date, end_date, course_length, 
    program_cost, graduation_rate, median_salary_180days) 
    VALUES (2, '1/1/2021', '6/30/2021', 'Self-paced',
    9540.00, 100.0, 45180)
    
INSERT INTO bootcamps_edu.Program_Date 
    (school_program_location_id, start_date, end_date, course_length, 
    program_cost, graduation_rate, median_salary_180days) 
    VALUES (3, '1/1/2021', '6/30/2021', '220',
    25000.00, 68.0, 75000)
    
INSERT INTO bootcamps_edu.Program_Date 
    (school_program_location_id, start_date, end_date, course_length, 
    program_cost, graduation_rate, median_salary_180days) 
    VALUES (4, '1/1/2021', '6/30/2021', '200',
    22500.00, 82.4, 50

In [34]:
# insert skills to the datatabase
columns = df.columns.values.tolist()

col = 0
for c in columns:
    col += 1

# 4 to 35 are skills

# connect to the database
conn = psycopg2.connect(**params2)

# insert into bootcamps_edu.Skill table
cursor = conn.cursor()
col = 0
for c in columns:
    if col >= 4 and col <= 35:
        cmd = f"INSERT INTO bootcamps_edu.Skill (skill) VALUES ('{c}')"
        cursor.execute(cmd)
    col += 1

conn.commit()
cursor.execute("select * from bootcamps_edu.Skill")
skill_lst = cursor.fetchall()
conn.commit()
conn.close()

print(skill_lst)



[(1, 'java'), (2, 'csharp_net'), (3, 'html'), (4, 'css'), (5, 'javascript'), (6, 'es6'), (7, 'jquery'), (8, 'nodejs'), (9, 'reactjs'), (10, 'redux'), (11, 'heroku'), (12, 'sql'), (13, 'mysql'), (14, 'postgress'), (15, 'mongodb'), (16, 'firebase'), (17, 'git'), (18, 'spring'), (19, 'api'), (20, 'applied_stats'), (21, 'python'), (22, 'r'), (23, 'regression'), (24, 'classification'), (25, 'clustering'), (26, 'time_series_analysis'), (27, 'anomaly_detection'), (28, 'language_processing'), (29, 'machine_learning'), (30, 'advanced_topics'), (31, 'story_telling_with_data'), (32, 'domain_expertise_development')]


In [35]:
# insert program_skills
# table bootcamps_edu.Program_Skill
#	id int PRIMARY KEY GENERATED BY DEFAULT AS IDENTITY
#	school_program_date_id int not null REFERENCES bootcamps_edu.Program_Date (id)
#	skill_id int not null REFERENCES bootcamps_edu.Skill (id)


# get all school, program, location from the original #csv
# get all school_id, program_id, location_id from the corresponding catalogs
# get school_id, program_id, location_id from school_program_location table using the ids above
# get school_program_location_id from school_program_location table
# for each row, get columns with skills and insert a row in program_skills for each skill

# iterating the columns, to create a list of column numbers and column names

col_list = []
sub_list = []
col_number = 0
for col in df.columns:
    sub_list = (col_number, col)
    if col_number >= 4 and col_number <= 35:
        col_list.append(sub_list)
    col_number += 1

col_df = pd.DataFrame(columns=('col_number', 'col_name'), data=col_list)

# connect to the database
conn = psycopg2.connect(**params2)
cursor = conn.cursor()

df_to_list = df.values.tolist()

for r in df_to_list:

    school = r[0]
    cmd = f"select * from bootcamps_edu.School where school_name = '{r[0]}'"
    cursor.execute(cmd)
    school_lst = cursor.fetchall()
    school_id = school_lst[0][0]

    location = r[1]
    cmd = f"select * from bootcamps_edu.Location where name = '{r[1]}'"
    cursor.execute(cmd)
    location_lst = cursor.fetchall()
    location_id = location_lst[0][0]

    program = r[2]
    cmd = f"select * from bootcamps_edu.Program where program_name = '{r[2]}'"
    cursor.execute(cmd)
    program_lst = cursor.fetchall()
    program_id = program_lst[0][0]

    cmd = f"""select * 
    from bootcamps_edu.School_Program_Location 
    where school_id = {school_id} and program_id = {program_id} and location_id = {location_id}"""
    cursor.execute(cmd)
    spl_lst = cursor.fetchall()
    spl_id = spl_lst[0][0]

    start_date = r[36]

    cmd = f"""select * 
    from bootcamps_edu.Program_Date
    where school_program_location_id = {spl_id}
    and start_date = '{start_date}'"""
    cursor.execute(cmd)

    program_date_lst = cursor.fetchall()
    program_date_id = program_date_lst[0][0]


    for i in range(4, 36):
        col_name = col_df[col_df['col_number'] == i]['col_name']

        skill_value = int(r[i])

        cmd = f"select * from bootcamps_edu.Skill where skill = '{col_name.values[0]}'"
        cursor.execute(cmd)
        skill_lst = cursor.fetchall()
        skill_id = skill_lst[0][0]

        cmd = f"""INSERT INTO bootcamps_edu.Program_Skill 
        (school_program_date_id, skill_id, skill_value)
        values ({program_date_id}, {skill_id}, {skill_value})
        """
        cursor.execute(cmd)

conn.commit()
conn.close()


